In [83]:
import pandas as pd

In [84]:
df = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = df[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [85]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ferran\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [87]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


In [88]:
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [89]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [90]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [91]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [92]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [93]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))
    

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


In [94]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


In [95]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [96]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.041*"polic" + 0.026*"charg" + 0.025*"case" + 0.025*"court" + 0.024*"death" + 0.021*"murder" + 0.017*"woman" + 0.017*"face" + 0.016*"alleg" + 0.013*"shoot"
Topic: 1 
Words: 0.047*"trump" + 0.025*"world" + 0.020*"open" + 0.018*"women" + 0.015*"island" + 0.013*"win" + 0.013*"return" + 0.012*"lose" + 0.012*"street" + 0.010*"sydney"
Topic: 2 
Words: 0.026*"live" + 0.023*"nation" + 0.018*"coronavirus" + 0.017*"restrict" + 0.015*"water" + 0.014*"life" + 0.013*"sentenc" + 0.012*"plan" + 0.011*"park" + 0.010*"council"
Topic: 3 
Words: 0.039*"queensland" + 0.026*"sydney" + 0.020*"bushfir" + 0.020*"crash" + 0.018*"adelaid" + 0.018*"die" + 0.015*"final" + 0.015*"perth" + 0.014*"miss" + 0.011*"break"
Topic: 4 
Words: 0.033*"year" + 0.020*"famili" + 0.018*"canberra" + 0.018*"melbourn" + 0.018*"tasmania" + 0.015*"jail" + 0.013*"work" + 0.013*"australian" + 0.013*"high" + 0.012*"tasmanian"
Topic: 5 
Words: 0.029*"govern" + 0.021*"coronavirus" + 0.019*"health" + 0.018*"victoria" + 0.

In [97]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.015*"coronavirus" + 0.014*"covid" + 0.009*"weather" + 0.008*"market" + 0.008*"street" + 0.008*"vaccin" + 0.008*"northern" + 0.007*"wall" + 0.007*"price" + 0.007*"australian"
Topic: 1 Word: 0.016*"countri" + 0.011*"hour" + 0.011*"restrict" + 0.009*"interview" + 0.009*"south" + 0.008*"queensland" + 0.008*"scott" + 0.007*"rain" + 0.007*"east" + 0.007*"coronavirus"
Topic: 2 Word: 0.011*"coronavirus" + 0.010*"care" + 0.008*"age" + 0.008*"turnbul" + 0.007*"lockdown" + 0.007*"children" + 0.007*"coal" + 0.006*"plead" + 0.006*"hobart" + 0.006*"histori"
Topic: 3 Word: 0.021*"news" + 0.014*"drum" + 0.013*"crash" + 0.010*"morrison" + 0.009*"monday" + 0.009*"tuesday" + 0.009*"peopl" + 0.009*"miss" + 0.008*"thursday" + 0.008*"search"
Topic: 4 Word: 0.018*"donald" + 0.012*"border" + 0.007*"korea" + 0.007*"kill" + 0.006*"outback" + 0.006*"russia" + 0.006*"biden" + 0.006*"novemb" + 0.005*"north" + 0.005*"china"
Topic: 5 Word: 0.012*"govern" + 0.009*"health" + 0.008*"elect" + 0.007*"bud

In [98]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6805880069732666	 
Topic: 0.029*"govern" + 0.021*"coronavirus" + 0.019*"health" + 0.018*"victoria" + 0.017*"help" + 0.015*"state" + 0.015*"school" + 0.013*"indigen" + 0.012*"fund" + 0.012*"communiti"

Score: 0.14080475270748138	 
Topic: 0.033*"year" + 0.020*"famili" + 0.018*"canberra" + 0.018*"melbourn" + 0.018*"tasmania" + 0.015*"jail" + 0.013*"work" + 0.013*"australian" + 0.013*"high" + 0.012*"tasmanian"

Score: 0.09102214127779007	 
Topic: 0.044*"australia" + 0.027*"elect" + 0.025*"donald" + 0.023*"kill" + 0.020*"coast" + 0.020*"australian" + 0.016*"border" + 0.014*"protest" + 0.013*"gold" + 0.012*"attack"

Score: 0.01251387968659401	 
Topic: 0.026*"live" + 0.023*"nation" + 0.018*"coronavirus" + 0.017*"restrict" + 0.015*"water" + 0.014*"life" + 0.013*"sentenc" + 0.012*"plan" + 0.011*"park" + 0.010*"council"

Score: 0.012513126246631145	 
Topic: 0.025*"news" + 0.023*"hous" + 0.017*"busi" + 0.017*"brisban" + 0.017*"peopl" + 0.016*"farmer" + 0.016*"time" + 0.014*"royal" + 0.0

In [99]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.32601499557495117	 Topic: 0.044*"australia" + 0.027*"elect" + 0.025*"donald" + 0.023*"kill" + 0.020*"coast"
Score: 0.20679618418216705	 Topic: 0.025*"news" + 0.023*"hous" + 0.017*"busi" + 0.017*"brisban" + 0.017*"peopl"
Score: 0.1838679015636444	 Topic: 0.033*"year" + 0.020*"famili" + 0.018*"canberra" + 0.018*"melbourn" + 0.018*"tasmania"
Score: 0.18326212465763092	 Topic: 0.041*"polic" + 0.026*"charg" + 0.025*"case" + 0.025*"court" + 0.024*"death"
Score: 0.016678139567375183	 Topic: 0.040*"covid" + 0.035*"coronavirus" + 0.032*"australia" + 0.030*"china" + 0.025*"test"
Score: 0.01667659915983677	 Topic: 0.029*"govern" + 0.021*"coronavirus" + 0.019*"health" + 0.018*"victoria" + 0.017*"help"
Score: 0.016676535829901695	 Topic: 0.047*"trump" + 0.025*"world" + 0.020*"open" + 0.018*"women" + 0.015*"island"
Score: 0.016676170751452446	 Topic: 0.022*"call" + 0.019*"rise" + 0.018*"victorian" + 0.015*"morrison" + 0.015*"scott"
Score: 0.016676127910614014	 Topic: 0.026*"live" + 0.023*"n